In [ ]:
#!pip install git+https://github.com/mlrun/mlrun@fix-remote-nuclio

### Subscribe to camera feeeds 
This function subscribes to a video feed and sends frames to 
an Iguazio stream

In [7]:
import nuclio

In [8]:
%run set_env.ipynb

In [9]:
%%nuclio env 
SHARD=0
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
IGZ_CONTAINER=${IGZ_CONTAINER} 
RAW_VIDEO_STREAM=${RAW_VIDEO_STREAM}
TAGGED_VIDEO_STREAM=${TAGGED_VIDEO_STREAM}
LOWRES_VIDEO_STREAM=${LOWRES_VIDEO_STREAM} 
CAMERA_LIST_TBL=${CAMERA_LIST_TBL} 

%nuclio: setting 'SHARD' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'IGZ_CONTAINER' environment variable
%nuclio: setting 'RAW_VIDEO_STREAM' environment variable
%nuclio: setting 'TAGGED_VIDEO_STREAM' environment variable
%nuclio: setting 'LOWRES_VIDEO_STREAM' environment variable
%nuclio: setting 'CAMERA_LIST_TBL' environment variable


In [10]:
%nuclio cmd -c pip install opencv-python v3io_frames requests

In [11]:
%%nuclio config
spec.minReplicas = 1
spec.maxReplicas = 1

%nuclio: setting spec.minReplicas to 1
%nuclio: setting spec.maxReplicas to 1


In [12]:
%nuclio mount /User /users/admin

mounting volume path /User as /users/admin


In [ ]:
# nuclio: start-code

In [ ]:
import threading
import cv2
from cv2 import VideoWriter_fourcc, VideoWriter
import sys
import os
import requests
import v3io_frames as v3f
import base64

In [ ]:
def stream_frame_write(context,cameraID,frame,shard):
    url = "http://v3io-webapi:8081/%s/%s/"% (os.getenv('IGZ_CONTAINER'),os.getenv('RAW_VIDEO_STREAM'))
    headers = {
            "Content-Type": "application/json",
            "X-v3io-function": "PutRecords",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }
    data = base64.b64encode(frame)
    payload = {
            "Records": [
                {
                    "Data":  data.decode('utf-8'),
                    "ShardId" : shard
                    }
                ]
          }
    r = requests.post(url, headers=headers,json=payload)   
    return r.text

In [ ]:
def _init_camera(context,
                cameraID: str,
                cameraURL:str,
                shard: int):
    
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('/User/igzcustomers/toyota/haarcascade_frontalface_default.xml')

    # To capture video from webcam.
    cap = cv2.VideoCapture(cameraURL)
    # To use a video file as input
    # cap = cv2.VideoCapture('filename.mp4')
    
    while True:

        # Display
        #cv2.imshow('img', img)

        fourcc = VideoWriter_fourcc(*'MPEG')
        #fourcc = VideoWriter_fourcc(*'MP4V')
        #out = VideoWriter(video_file, fourcc, 60.0, (640, 480))
        running_size=0
        while (cap.isOpened()):
            ret, img = cap.read()
            ret, buffer = cv2.imencode('.jpg', img)
            #try:
            if True:
                r = stream_frame_write(context,cameraID,buffer.tobytes(),shard)
                context.logger.debug(r)
            #except:
            #    context.logger.error("Failed to save frame to shard: %s"% str(shard))
            #save_count_and_image(face_count,'105lf')


        # Stop if escape key is pressed
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    # Release the VideoCapture object
    cap.release()
    

In [ ]:
def get_cameras_list():
    client = v3f.Client('framesd:8081',container=os.getenv('IGZ_CONTAINER'))
    df=client.read('kv',os.getenv('CAMERA_LIST_TBL'))
    return df

In [ ]:
def handler(context, event):
    return ""

In [ ]:
def init_context(context):
    cameras_list = get_cameras_list()
    for index, row in get_cameras_list().iterrows():
        if row['active'] and row['feed_method'] == 'pull':
            thread = threading.Thread(target=_init_camera, args=(context,index,row['url'],row['shard']))
            thread.start()


In [ ]:
# nuclio: end-code

In [13]:
import os

In [14]:
# nuclio: ignore
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('cameras-pull', kind='nuclio')

In [15]:
# set the API/trigger, attach the home dir to the function
fn.with_http(workers=1).apply(mount_v3io())
fn.spec.build.commands = ['pip install opencv-python v3io_frames requests']
fn.spec.build.base_image = 'mlrun/mlrun'
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.set_env('V3IO_ACCESS_KEY',os.getenv('V3IO_ACCESS_KEY'))
fn.set_env('IGZ_CONTAINER',os.getenv('IGZ_CONTAINER'))
fn.set_env('RAW_VIDEO_STREAM',os.getenv('RAW_VIDEO_STREAM'))
fn.set_env('CAMERA_LIST_TBL',os.getenv('CAMERA_LIST_TBL'))
fn.apply(mount_v3io())

In [ ]:
# nuclio: ignore
addr = fn.deploy(project='objectrecognition')

[mlrun] 2020-08-14 14:47:15,863 deploy started
[nuclio] 2020-08-14 14:47:16,951 (info) Build complete


In [ ]:
for index, row in get_cameras_list().iterrows():
        print(row.keys(),index)
        
